In [113]:
import os
import requests
from dotenv import load_dotenv
import bs4
from bs4 import BeautifulSoup
import lxml
from IPython.display import Markdown, display
from openai import OpenAI

In [115]:
openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [116]:
headers={
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
} 
#a dictionary named header so that we can grab same html code as the user ,and also to avoid blocks,captcha and error403

In [119]:
class Website:
    def __init__(self,url):
        self.url=url
        response= requests.get(url,headers=headers,timeout=30)
        soup=BeautifulSoup(response.content,'lxml')
        self.title=soup.title.string if soup.title else "No title found"#scraping the content
        for irrelevant in soup.body(["script", "style", "img", "input"]):#cleaning the content
            irrelevant.decompose()
            #using .get_text() method of Beautiful soup
        self.text = soup.body.get_text(separator="\n", strip=True)#creating space between different lines and removing leading whitespaces by strip=true

In [121]:
gossip= Website("https://timesofindia.indiatimes.com/entertainment")
print(gossip.title)
print(gossip.text)

Latest and Trending Entertainment News, Celebrity News, Movie News, Breaking News | Entertainment - Times of India
Sign In
TOI
Go to
TOI
Etimes
home
cinema
news
movie reviews
movie listings
box office
anime
previews
did you know
videos
showtimes
blogs
awards
News
entertainment
Trending
Javed Akhtar
Diljit Dosanjh
Jaideep Ahlawat
Karisma Kapoor
Gauri Khan
Blake Lively
Trisha Krishnan
Kuberaa Box Office Collection
Sitaare Zameen Par Box Office Collection
Housefull 5
Kuberaa Movie Review
Sitaare Zameen Par Movie Review
Javed Akhtar
Diljit Dosanjh
Jaideep Ahlawat
Karisma Kapoor
Gauri Khan
Blake Lively
Trisha Krishnan
Kuberaa Box Office Collection
Sitaare Zameen Par Box Office Collection
Housefull 5
Kuberaa Movie Review
Sitaare Zameen Par Movie Review
Javed Akhtar
Diljit Dosanjh
Jaideep Ahlawat
Karisma Kapoor
Gauri Khan
Blake Lively
Trisha Krishnan
Kuberaa Box Office Collection
Sitaare Zameen Par Box Office Collection
Housefull 5
Kuberaa Movie Review
Sitaare Zameen Par Movie Review
Sudhansh

In [123]:
system_prompt = """
You are a stylish and culturally aware assistant who specializes in summarizing and discussing fashion trends, celebrity style, entertainment news, and television gossip.

You stay updated on Hollywood, Bollywood, and the television world—including celebrity rumors, drama, reality TV updates, show recaps, and behind-the-scenes stories.

When summarizing content, be engaging, concise, and insightful. Focus on what's trending, who's wearing what, and what everyone is talking about in fashion and entertainment. Maintain a fun yet informative tone, like a pop culture expert writing for a lifestyle magazine.

If content includes TV gossip, highlight key rumors, casting updates, fan reactions, and noteworthy moments from popular shows.
"""

In [125]:
def user_prompt_for(website):
    user_prompt = f"""The following text is extracted from a website titled: "{website.title}".

Please analyze this content and provide a short and engaging summary in **Markdown format**.

If the page contains:
- 🧵 Fashion trends: mention standout styles, designers, or events.
- 🗣️ TV gossip: highlight any drama, casting news, or fan reactions.
- 🎬 Celebrity updates (Hollywood/Bollywood): include relevant quotes, fashion moments, or event mentions.
- 📺 Show recaps: summarize what happened and any major twists.

Keep the summary clear, fun, and informative. Use bullet points if multiple themes appear. If there is no meaningful content, say: *“No relevant summary could be generated.”*

Website Content:
{website.text}
"""
    return user_prompt

In [127]:
print(user_prompt_for(gossip))

The following text is extracted from a website titled: "Latest and Trending Entertainment News, Celebrity News, Movie News, Breaking News | Entertainment - Times of India".

Please analyze this content and provide a short and engaging summary in **Markdown format**.

If the page contains:
- 🧵 Fashion trends: mention standout styles, designers, or events.
- 🗣️ TV gossip: highlight any drama, casting news, or fan reactions.
- 🎬 Celebrity updates (Hollywood/Bollywood): include relevant quotes, fashion moments, or event mentions.
- 📺 Show recaps: summarize what happened and any major twists.

Keep the summary clear, fun, and informative. Use bullet points if multiple themes appear. If there is no meaningful content, say: *“No relevant summary could be generated.”*

Website Content:
Sign In
TOI
Go to
TOI
Etimes
home
cinema
news
movie reviews
movie listings
box office
anime
previews
did you know
videos
showtimes
blogs
awards
News
entertainment
Trending
Javed Akhtar
Diljit Dosanjh
Jaideep Ahl

In [129]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [146]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "llama3.2",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [ ]:
summarize("https://timesofindia.indiatimes.com/entertainment")

In [139]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [ ]:
display_summary("https://timesofindia.indiatimes.com/entertainment")